# 신뢰구간 (using NHANES data)

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [2]:
da = pd.read_csv('nhanes_2015_2016.csv')

In [3]:
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


# 1) One proportion    
## Question : 남성과 여성, 각각의 흡연자 비율은 어떻게 될까?

 주어진 샘플 데이터로 모집단의 비율 측정   
 남성과 여성, 각각의 흡연자 비율을 신뢰도 95% 수준으로 추정(estimate)   
 SMQ020 = 흡연 유무 (1: 흡연, 2: 비흡연, 7 or 9: 기타)   
 RIAGENDR = 성별 (1: 남자, 2: 여자)

### 신뢰 구간 :

$$Best\ Estimate \pm Margin\ of\ Error$$

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

#### 데이터 처리

In [4]:
da.SMQ020.unique()

array([1, 2, 7, 9], dtype=int64)

In [5]:
da.SMQ020.value_counts()

2    3406
1    2319
9       8
7       2
Name: SMQ020, dtype: int64

In [6]:
da.RIAGENDR.unique()

array([1, 2], dtype=int64)

In [7]:
da.RIAGENDR.value_counts()

2    2976
1    2759
Name: RIAGENDR, dtype: int64

In [8]:
da['SMQ020x'] = da.SMQ020.replace({1: 'Yes', 2: 'No', 7: np.nan, 9: np.nan})
da['RIAGENDRx'] = da.RIAGENDR.replace({1: 'Male', 2: 'Female'})
dx = da[['SMQ020x', 'RIAGENDRx']].dropna()

In [9]:
male = da[da['RIAGENDRx']=='Male']['SMQ020x'].value_counts()

In [10]:
female = da[da['RIAGENDRx']=='Female']['SMQ020x'].value_counts()

In [11]:
table = {'Male': male, 'Female': female}
pd.DataFrame(data=table)

,Male,Female
No,1340,2066
Yes,1413,906


In [12]:
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [13]:
dz = dx.groupby(dx.RIAGENDRx).agg({'SMQ020x': [lambda x: np.mean(x=='Yes'), np.size]})
dz.columns=['Proportion', 'Total_n']

In [14]:
dz

,Proportion,Total_n
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


#### 남성, 여성 샘플의 흡연 proportion, sample size, standard error 계산

In [15]:
# female
p_f = dz.Proportion.Female
n_f = dz.Total_n.Female
se_female = np.sqrt(p_f*(1-p_f)/n_f)
se_female

0.008444152146214435

In [16]:
# male
p_m = dz.Proportion.Male
n_m = dz.Total_n.Male
se_male = np.sqrt(p_m*(1-p_m)/n_m)
se_male

0.009526078653689868

#### 신뢰도 95%의 신뢰 구간 계산
sample size > 30 : large enough   
t-multiplier = 1.96

In [17]:
# 여자
lcb_f = p_f - 1.96*np.sqrt(p_f*(1-p_f)/n_f)
ucb_f = p_f + 1.96*np.sqrt(p_f*(1-p_f)/n_f)
print(lcb_f, ucb_f)

0.288294683866098 0.32139576027925865


In [18]:
# 남자
lcb_m = p_m - 1.96*np.sqrt(p_m*(1-p_m)/n_m)
ucb_m = p_m + 1.96*np.sqrt(p_m*(1-p_m)/n_m)
print(lcb_m, ucb_m)

0.49458714955108174 0.531929377873546


#### statsmodel을 통해 계산

In [19]:
# 여자
sm.stats.proportion_confint(906, 906+2066)  

(0.2882949879861214, 0.32139545615923526)

In [20]:
# 남자
sm.stats.proportion_confint(1413, 1413+1340)  

(0.49458749263718593, 0.5319290347874418)

## 해석(conclusion)

**신뢰도 95%로**, 모집단에서의 여성 흡연자 비율은 28.8% - 32.1%로 **추정된다.**   
**신뢰도 95%로**, 모집단에서의 남성 흡연자 비율은 49.5% - 53.2%로 **추정된다.**   
   
   신뢰도 95%라는 의미는, 수많은 샘플을 같은 방식과 계산으로 신뢰 구간들을 추정했을 때, 수많은 신뢰 구간들 중 95%가 '모집단의 비율'을 포함하고 있다는 의미이다.   
        
   


#  

# 2) *Comparing* two independent proportions   
## Question : 남성과 여성의 흡연자 비율의 *차이(difference)*는 어떻게 될까?


$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{(SE_{\ 1})^2 + (SE_{\ 2})^2}$$

In [21]:
se_diff = np.sqrt(se_female**2 + se_male**2)
se_diff

0.012729881381407434

In [22]:
diff = dz.Proportion.Female - dz.Proportion.Male
lcb = diff - 2*se_diff
ucb = diff + 2*se_diff
print(lcb, ucb)

-0.2338728044024504 -0.18295327887682067


## 해석(conclusion)

**신뢰도 95%로**, 전체 여성 흡연자 비율은 남성 흡연자 비율보다 0.23% - 0.18% 포인트 **낮은 것으로 추정된다.**  
   
'파라미터의 차이'를 추정할 때, 95% 신뢰 구간이 0을 포함하지 않는다면 '파라미터의 차이'가 유의 수준 0.05에서 통계적으로 유의하다고 볼 수 있다.

   ### 만약 신뢰 구간 내 0이 포함되면 어떻게 될까?
   신뢰 구간 내 0이 포함된다면, 여성 흡연자 비율과 남성 흡연자 비율의 차이가 0이 된다는 것은 **일반적이다.**   
   다른 추가적인 테스트가 필요하다.   
   귀무가설을 '두 그룹 간 비율의 차이가 0이다.'라고 설정하고 가설 검정을 한다. 만약 p-value가 설정한 유의 수준 value(일반적으로 0.05)보다 클 경우 귀무가설을 기각할 증거가 부족하고, 두 그룹 간 차이가 0이라고 주장하는 귀무가설을 지지해야한다.     
      
   

# 


# 3) One Mean
## Question : 남성과 여성 각각의 평균 BMI는 얼마?

BMXBMI: body mass inde (BMI) 체질량 지수

In [23]:
da.groupby('RIAGENDRx').agg({'BMXBMI': [np.mean, np.std, np.size]})

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

In [24]:
size_f = 2976
size_m = 2759
sem_female = 7.753 / np.sqrt(size_f)
sem_male = 6.253 / np.sqrt(size_m)
print(sem_female, sem_male)

0.14211938534506902 0.119045388988243


In [25]:
#여성
lcb_female = 29.94 - 1.96 * 7.753 / np.sqrt(size_f)
ucb_female = 29.94 + 1.96 * 7.753 / np.sqrt(size_f)
print(lcb_female, ucb_female)

29.661446004723665 30.218553995276338


In [26]:
#남성
lcb_male = 28.78 - 1.96 * 6.253 / np.sqrt(size_m)
ucb_male = 28.78 + 1.96 * 6.253 / np.sqrt(size_m)
print(lcb_male, ucb_male)

28.546671037583046 29.013328962416956


#### statsmodel을 이용한 계산

In [27]:
female_bmi = da.loc[da.RIAGENDRx=="Female", "BMXBMI"].dropna()
sm.stats.DescrStatsW(female_bmi).zconfint_mean()

(29.659875498090155, 30.22001580625768)

In [28]:
male_bmi = da.loc[da.RIAGENDRx=="Male", "BMXBMI"].dropna()
sm.stats.DescrStatsW(male_bmi).zconfint_mean()

(28.543010471866516, 29.013133751827375)

## 해석(conclusion)

**신뢰도 95%로**, 전체 여성 BMI 평균은 29.66 - 30.22로 **추정된다.**   
**신뢰도 95%로**, 전체 남성 BMI 평균은 28.54 - 29.01로 **추정된다.**  

# 

# 4) *Difference* between two means
## Question : 남성과 여성의 평균 BMI 차이는 얼마?

In [29]:
sem_diff = np.sqrt(sem_female**2 + sem_male**2)
sem_diff

0.18539073420811059

In [30]:
bmi_diff = 29.94 - 28.78
lcb = bmi_diff - 1.96*sem_diff
ucb = bmi_diff + 1.96*sem_diff
(lcb, ucb)

(0.7966341609521034, 1.523365839047897)

## 해석(conclusion)

**신뢰도 95%로**, 여성의 BMI 평균이 남성의 BMI 평균보다 0.8 - 1.52 **높은 것으로 추정된다.**   
   
신뢰 구간이 0을 포함하지 않으므로, 차이는 유의하다.